In [1]:
# Importações
import pandas as pd
import numpy as np
from datetime import datetime
from difflib import get_close_matches
import re
from thefuzz import process, fuzz
from unidecode import unidecode

In [2]:
# Visualização da tabela suja (estudos iniciais)
df = pd.read_csv('../vendas_modificado.csv')
df_mod = df.copy()
df.head(150)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,3/20/2021,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"R$ 16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,10/30/2020,9:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"R$ 3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,6/9/2021,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"R$ 1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,6/4/2022,8:41:23,LEVI RIBEIRO AMORIM,Carvão,"R$ 8,74",4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,5/4/2019,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,"R$ 9,48",2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,10847,10/17/2022,21:51:08,MATHEUS GONÇALVES DOS SANTOS,Sabão em Pó,"R$ 6,84",1,18.84,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,12.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Brilhante
146,27900,10/22/2018,21:53:30,LUCAS HENRIQUE CAÇADOR,Café,"R$ 10,57",2,39.14,Aguardando Pagamento,Santo André,SP,Brasil,09000-000,18.0,Cartão de Crédito,HENRICO VICTOR ALVES,Pilão
147,44827,10/1/2020,15:16:00,JOÃO SOUZA RONCETE,Shampoo,"R$ 11,34",12,136.08,Pagamento Confirmado,Ritápolis,MG,Brasil,36335-000,0.0,Pix,LUCAS VITOR FAÇANHA NEVES,Seda
148,16077,11/12/2022,15:52:53,RAFAEL HUGO CAZULA PEREIRA,Desinfetante,"R$ 5,5",3,16.50,Pagamento Confirmado,Ubá,MG,Brasil,36500-000,0.0,Transferência Bancária,HENRICO VICTOR ALVES,Lysoform


### **PROBLEMA:** Problemas de Formatação (Datas e Horas)
### **SOLUÇÃO:** Padronizar datas e horas

In [3]:
df_mod['data'] = pd.to_datetime(df_mod['data'], errors='coerce').dt.strftime('%Y-%m-%d')


In [4]:
df_mod['hora'] = pd.to_datetime(df_mod['hora'], errors='coerce').dt.strftime('%H:%M:%S')

C:\Users\Jakarta\AppData\Local\Temp\ipykernel_14192\629907975.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_mod['hora'] = pd.to_datetime(df_mod['hora'], errors='coerce').dt.strftime('%H:%M:%S')


### **PROBLEMA:** Problemas de Formatação e Valores Inconsistentes em "valor"
### **SOLUÇÃO:** Converter valores monetários de string para float


In [5]:
def normalizar_valor(valor):
    if pd.isna(valor): return np.nan
    valor_str = str(valor).replace('R$', '').replace('.', '').replace(',', '.').strip()
    try:
        return round(float(valor_str), 2)
    except:
        return np.nan

df_mod['valor'] = df_mod['valor'].apply(normalizar_valor)

### **PROBLEMA:** Tipos de Dados incorretos nas colunas numéricas
### **SOLUÇÃO:** Converter com segurança para os tipos corretos

In [6]:
colunas_numericas = ['quantidade', 'total', 'frete']
for col in colunas_numericas:
    df_mod[col] = pd.to_numeric(df_mod[col], errors='coerce')

df_mod['quantidade'] = df_mod['quantidade'].round().astype(int)

### **PROBLEMA:** Valores Inconsistentes na coluna "total"
### **SOLUÇÃO:** Recalcular corretamente o total


In [7]:
# Corrigir tipo da quantidade (int)
def corrigir_total(df):
    df_corrigido = df.copy()
    df_corrigido['total'] = (df_corrigido['valor'] * df_corrigido['quantidade'] + df_corrigido['frete']).round(2)
    return df_corrigido

df_mod = corrigir_total(df_mod)

### **PROBLEMA:** Dados Faltantes nas colunas críticas
### **SOLUÇÃO:** Preencher com valores padrão seguros

In [8]:
colunas_criticas = ['valor', 'quantidade', 'total', 'frete', 'cliente', 'vendedor', 'cep']
for col in colunas_criticas:
    if df_mod[col].dtype == object:
        df_mod[col] = df_mod[col].fillna('Desconhecido').str.strip()
    else:
        df_mod[col] = df_mod[col].fillna(0)

### **PROBLEMA:** Problemas de Padronização (Produtos com variações)
### **SOLUÇÃO:** Limpeza de caracteres e agrupamento por similaridade textual

In [9]:
def limpar_caracteres_especiais(df, column='produto'):
    df[column] = df[column].astype(str).apply(lambda x: re.sub(r'[^\w\s]', '', x).strip())
    return df

def normalizar_nomes_produtos(df, column='produto', threshold=60):
    produtos_unicos = df[column].dropna().unique()
    mapping = {}
    for produto in produtos_unicos:
        similares = [p for p in produtos_unicos if fuzz.ratio(produto, p) >= threshold]
        padrao = df[df[column].isin(similares)][column].mode()[0]
        for p in similares:
            mapping[p] = padrao
    df[column] = df[column].map(mapping)
    return df

df_mod = limpar_caracteres_especiais(df_mod, 'produto')
df_mod = normalizar_nomes_produtos(df_mod, 'produto')

### **PROBLEMA:** Problemas de Validação (CEP)
### **SOLUÇÃO:** Validar, limpar e aplicar formato 00000-000

In [10]:
def normalizar_cep(cep):
    if pd.isna(cep): return '00000-000'
    cep = re.sub(r'\D', '', str(cep))
    if len(cep) != 8 or cep == '00000000':
        return '00000-000'
    return f'{cep[:5]}-{cep[5:]}'

df_mod['cep'] = df_mod['cep'].apply(normalizar_cep)

# **Remoção de duplicatas**

In [11]:
df_mod.drop_duplicates(inplace=True)

# **Validação final de limpeza**

In [12]:
assert df_mod.isnull().sum().sum() == 0, "Ainda há valores nulos"
assert df_mod['quantidade'].dtype == int, "'quantidade' não é int"
assert df_mod['valor'].dtype in [float, np.float64], "'valor' não é float"
assert df_mod['total'].dtype in [float, np.float64], "'total' não é float"

# **Exportação final para CSV**

In [13]:
df_mod.to_csv("../limpeza_final.csv", index=False)
print("Arquivo exportado como 'limpeza_final.csv'")

Arquivo exportado como 'limpeza_final.csv'
